In [ ]:
import json
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.express as px

from eda_utils import (
    read_scip_log,
    format_log_df,
    operations_plots,
    objective_function_plots,
    get_month_difference,
    get_revenue_before_eow,
)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

%load_ext autoreload
%autoreload 2


In [ ]:
experiment = "2019_24periods_12eow_real_prices"

In [ ]:
with open("../lp_logs/experiments_results.json") as json_file:
    experiment_results = json.load(json_file)
with open("../lp_logs/params.json") as json_file:
    PARAMS = json.load(json_file)

In [ ]:
for experiment, value in experiment_results.items():
    Log_df = read_scip_log(f"../lp_logs/{experiment}.log")
    Log_df = format_log_df(Log_df)

    periods_before_eow = get_month_difference(
        experiment_results[experiment]["fecha_inicio"],
        experiment_results[experiment]["fecha_fin_ejercicio"],
    )

    experiment_results[experiment]["lp_revenue_before_eow"] = get_revenue_before_eow(
        Log_df, periods_before_eow, PARAMS, experiment_results, experiment
    )

    experiment_results[experiment]["lp_revenue_eow"] = Log_df[
        "impact_on_obj_func"
    ].sum()

In [ ]:
experiment_results["2019_24periods_12eow_real_prices"]

In [ ]:
df_table = pd.DataFrame()
for experiment, value in experiment_results.items():
    series = (
        pd.Series(
            {
                "fecha_inicio": value["fecha_inicio"],
                "fecha_fin_ejercicio": value["fecha_fin_ejercicio"],
                "periodos_modelo": value["periodos_modelo"],
                "fix_prices": value["fix_prices"],
                "lp_revenue_before_eow": value["lp_revenue_before_eow"]
                - value["lp_stock_history_cost"],
                "lp_revenue_eow": value["lp_revenue_eow"]
                - value["lp_stock_history_cost"],
                "business_grand_total": value["business_results"]["grand_total"],
            }
        )
        .to_frame()
        .T
    )
    df_table = pd.concat([df_table, series])

In [ ]:
df_table["impact"] = (
    df_table["lp_revenue_before_eow"] - df_table["business_grand_total"]
) / df_table["business_grand_total"]
df_table["impact"] = df_table["impact"].apply(round, args=[2]) * 100
df_table

In [ ]:
operations_plots(
    Log_df,
    periodos_modelo=experiment_results[experiment]["periodos_modelo"],
    periods_before_eow=periods_before_eow,
)

objective_function_plots(
    Log_df,
    periodos_modelo=experiment_results[experiment]["periodos_modelo"],
    periods_before_eow=periods_before_eow,
)